In [5]:
import os
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns

from datetime import datetime

### Definição de caminhos e diretórios

In [6]:
raw_path    = "data/raw"
bronze_path = "data/bronze"
silver_path = "data/silver"

### Helper Functions

### Criando os datasets

In [13]:
# Grava dataframe em arquivo .csv
def df_to_csv( df, path, mode ):
    with open( path, mode ) as csv_file:
        df.to_csv( csv_file, index = False )

In [16]:
def silver_datasets_generation():  
    file_list = []
    df_silver = pd.DataFrame()
    
    if os.path.exists( bronze_path ):
        file_list = os.listdir( bronze_path )

        for file in file_list[0:1]:
            if '.csv' in file:
                try:
                    # loadin the bronze dataset into df_silver
                    df_silver = pd.read_csv( f"{bronze_path}/{file}" )
                    # sorting the df_silver by the open time
                    df_silver.sort_values( "Open_time" )
                    # dropping the ignore column
                    df_silver.drop( "Ignore", axis = 1, inplace = True )
                    # reset index 
                    df_silver.reset_index( inplace = True, drop = True )
                    
                    # converting Open_time and Close_time columns to datetime
                    df_silver[ 'Open_time' ] = df_silver[ 'Open_time' ].apply( lambda x: datetime.fromtimestamp( np.round( x/1000, 0 ) ) )
                    df_silver[ 'Close_time' ] = df_silver[ 'Close_time' ].apply( lambda x: datetime.fromtimestamp( np.round( x/1000, 0 ) ) )
                    
                    # Adding the mean price column
                    df_silver[ "Mean"] = df_silver[["Open","High","Low","Close"]].sum( axis = 1 ) / 4
                    df_silver = df_silver [["Open_time","Open","High","Low","Close","Mean","Volume","Close_time","Quote_asset_volume","Number_of_trades","Taker_buy_base_asset_volume","Taker_buy_quote_asset_volume"]]
                    
                except Exception as ex:
                    print( f"ERROR: {ex}" )
                
                
                
    return df_silver
df_silver = silver_datasets_generation()     

## Incluindo os Indicadores

### SMA (Simple Moving Average)

In [ ]:
# Simple Moving Average 
def SMA(data, ndays, _name): 
    SMA = pd.Series(data['Close'].rolling(ndays).mean(), name = _name) 
    data = data.join(SMA) 
    return data

In [ ]:
df_silver = SMA( df_silver, 50, 'LT_SMA')
df_silver.head()

,Open_time,Open,High,Low,Close,Mean,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume
0,2021-06-30 21:00:00,57.549,57.645,56.744,56.744,57.17050,33446.870,2021-06-30 21:05:00,1.908515e+06,2614,14504.105,827697.106135
1,2021-06-30 21:05:00,56.741,56.769,56.239,56.259,56.50200,21475.548,2021-06-30 21:10:00,1.214538e+06,1941,8971.813,507724.090608
2,2021-06-30 21:10:00,56.243,56.500,56.214,56.380,56.33425,11521.704,2021-06-30 21:15:00,6.494414e+05,1173,6580.538,371010.849112
3,2021-06-30 21:15:00,56.424,56.661,56.228,56.469,56.44550,12765.234,2021-06-30 21:20:00,7.209205e+05,1136,7910.369,446941.910521
4,2021-06-30 21:20:00,56.484,56.912,56.479,56.617,56.62300,17825.950,2021-06-30 21:25:00,1.011476e+06,1259,8613.917,488766.239262


### Gravando os silvers datasets

In [20]:
# Precisa revisar
try:
    if os.path.exists( silver_path ):
        # pass
        df_to_csv( df_silver, f"{silver_path}/{file}", 'a' )
    
    else:
        os.mkdir( silver_path )
        df_to_csv( df_silver, f"{df_silver}/{file}", 'a' )

except Exception as ex:
    print( f"ERROR: {ex}" )

ERROR: name 'file' is not defined
